In [42]:
# %matplotlib ipympl 
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import boxcox
from scipy.stats import yeojohnson

In [43]:
# load all data into dataframe
def load_data(path, file_names, aliases):
    dates = {}
    for data_set_idx in range(len(data_files)):
        cur_alias = aliases[data_set_idx]
        with open(path + data_files[data_set_idx] + '.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            spamreader.__next__()
            for row in spamreader:
                try:
                    cur_date = datetime.datetime.strptime(row[0], '%m/%d/%Y')

                except Exception as e: 
                   continue
                if not cur_date in dates:
                    dates[cur_date] = {}
#                     # need to generalize here
#                 if data_set_idx == 0 or data_set_idx == 2:
#                     dates[cur_date][cur_alias] = float(row[4])
#                 elif data_set_idx == 1 or data_set_idx == 3 or data_set_idx == 4 or data_set_idx == 5:
                try:
                    dates[cur_date][cur_alias] = float(row[1])
                except:
                    print(row[1])
                    print(cur_alias)
                    print(row)
                    

    df = pd.DataFrame.from_dict(dates, orient='index')
    # df.columns = aliases
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Date'})
    df = df.sort_values('Date')
    df = df.reset_index(drop=True)
    return df


In [44]:
# not_null = df.query(baseline_asset + ".notnull()")
# not_null.reset_index(drop = True, inplace = True)

In [45]:
def load_div_data(asset_list, file_list):
    div_data = {}
    for asset, file_path in zip(asset_list, file_list):
        # Initialize data structure for the asset
        div_data[asset] = {"payment_date": set(), "ex_date": set(), "amount": {}}
        
        # Read CSV file into DataFrame
        if file_path != None:
            df = pd.read_csv(file_path, delimiter=',', header=0)
            
            # Iterate over rows in the DataFrame
            for index, row in df.iterrows():
                # Extract relevant data
                ex_date = row["Ex/EFF Date"]
                cash_amount = row["Cash Amount"]
                payment_date = row["Payment Date"]
                
                # Update div_data with extracted data
                div_data[asset]["ex_date"].add(ex_date)
                div_data[asset]["payment_date"].add(payment_date)
                div_data[asset]["amount"][ex_date] = cash_amount
            
    return div_data


def get_x_days_ret(asset, df, div_data, distance, idx):
    start_idx = 0
    end_idx = 0
    if distance < 0:
        distance = abs(distance)
        start_idx = idx - distance
        end_idx = idx + 1
    else:
        start_idx = idx
        end_idx = idx + distance + 1
        
    num_shares = 1
    dollars = 0
    for i in range(start_idx, end_idx):
        if df.iloc[i]["Date"] in div_data["payment_date"]:
            num_shares += dollars / df.iloc[i][asset]
            dollars = 0
        if df.iloc[i]["Date"] in div_data["ex_date"]:
            dollars += div_data["amount"][df.iloc[i]["Date"]] * num_shares
    final_val = num_shares * df.iloc[end_idx - 1][asset] + dollars
    start_val = df.iloc[start_idx][asset]
    return (final_val - start_val) / start_val
            
            
            
    

def add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            if idx > days_out: # check for enough data
                # get percent change
                time_period_change = get_x_days_ret(asset, df, div_data[asset], -days_out, idx)  
                cors[alias_idx].append(time_period_change)
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance][asset]):
                time_period_change = get_x_days_ret(asset, df, div_data[asset], pred_distance, idx)  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [46]:
div_data = load_div_data(["re"], ["IYRDividends.csv"])

FileNotFoundError: [Errno 2] No such file or directory: 'IYRDividends.csv'

In [48]:
# adds correlation metrix to dataframe
def add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            cur_price = row[asset]
            if idx > days_out: # check for enough data
                # get percent change
                last_time_period = df.loc[idx - days_out - 1].at[asset]
                time_period_change = (cur_price - last_time_period)/last_time_period
                cors[alias_idx].append(time_period_change)           
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance].at[asset]):
                fut_val = df.iloc[idx + pred_distance].at[asset]
                time_period_change = (fut_val - cur_price)/cur_price  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [49]:
def add_pred_differences(pred_distance, baseline_asset, assets, df):
    for idx, asset in enumerate(assets):
#       for idx2, asset2 in enumerate(assets[idx + 1: ]): if you want all differences
        if asset != baseline_asset:
            change_asset = df[asset + "_fut_" + str(pred_distance) + "dys"]
            change_baseline = df[baseline_asset + "_fut_" + str(pred_distance) + "dys"]
            diff = change_asset - change_baseline 
            df.insert(df.shape[1], asset + "_" + baseline_asset + "_" + str(pred_distance) + "dys_diff", diff, True)

In [50]:
# not_null.iloc[5700:5750,[0, 7,8, 9, 10,11,12,13,14,15,16,17,18,19,20]]

In [51]:
# not_null.iloc[-160:,0: 10]

In [52]:
# not_null.columns

In [53]:
# #use sklearn.preprocessing.PowerTransformer instead

# plt.figure()

# column_name = 'sp_20_dys'
# column = not_null[column_name] 
# column = column[~np.isnan(column)]
# print(column)
# # column += np.array([1 for i in range(len(column))])
# # print(column)
# plt.figure()
# plt.hist(column , color = 'red', bins = 500, density=True)
# mean = np.mean(column)
# std = np.std(column)
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))


# plt.figure()
# plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# mean = np.mean(yeojohnson(column)[0])
# std = np.std(yeojohnson(column)[0])
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])) , color = 'red', bins = 500, density=True)
# # mean = np.mean(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))
# # std = np.std(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))



# # column += np.array([1 for i in range(len(column))])
# # column = np.log(column)
# # plt.figure()
# # plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# # mean = np.mean(yeojohnson(column)[0])
# # std = np.std(yeojohnson(column)[0])
# # print(mean)
# # print(std)
# # x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(sp_not_null['sp_fut_2wks'], color = 'red', bins = 500)
# # plt.hist(sp_not_null['re_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['bnd_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['gld_fut_2wks'], color = 'yellow', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['eu_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['jp_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# plt.show()

In [54]:
# fig = plt.figure()
# ax = fig.add_subplot(projection = '3d')

# ax.scatter(not_null["sp_last_month"], not_null["re_last_month"], not_null["re_sp_2wk_diff"])
# ax.set_xlabel('sp_last_month')
# ax.set_ylabel('re_last_month')
# ax.set_zlabel('re_sp_2wk_diff')
# plt.show()

In [55]:
def get_rvs(baseline, df, aliases, pred_distance, print_mats=False):
    rvs = {}
    
    valid_cols = []
    for col in df.columns:
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            valid_cols.append(col)
    valid_cols.append(None)
    
    for asset in aliases:
        if asset != baseline:
            valid_cols[-1] = (asset + "_" + baseline + "_" + str(pred_distance) + "dys_diff")

            cov_mat = df[valid_cols]
            cov_matrix = pd.DataFrame.cov(cov_mat)
            cov_mat = cov_mat.cov()
            cov_mat = cov_mat.to_numpy()
            if print_mats:
                print(asset)
                print(cov_matrix)


            # means of values
            means = []
            for col in valid_cols:
                means.append(np.mean(df[col]))
            if print_mats:
                print(means)

            rv = multivariate_normal(mean=means, cov=cov_mat, allow_singular=True)
            rvs[asset] = rv
    return rvs
        


def predict(asset, baseline, rv, inputs, get_plots=False, do_print=False, get_50_pt = False):
    START = -.22
    STOP = .22
    INCREMENT = .00005

    probs = []
   
    x = np.arange(START, STOP, INCREMENT)
    inputs.append(None)
    for val in x:
        # make an array with all the current values
        # insert past month performance
        #"sp", "re", "bnd", "eu", "jp", "gld", future difference
        inputs[-1] = val
        probs.append(rv.pdf(inputs))


    cdf = []
    for idx in range(x.size - 1):
        cur_prob = probs[idx]
        next_prob = probs[idx + 1]
        rieman_sum = min(cur_prob, next_prob) * INCREMENT
        rieman_sum += max(cur_prob, next_prob) - min(cur_prob, next_prob) * INCREMENT / 2
        if len(cdf) > 0:
            cdf.append(rieman_sum + cdf[-1])
        else:
            cdf.append(rieman_sum)
                                 
    if get_plots:
        fig1 = plt.figure()
        ax = fig1.add_subplot(111)
        plt.title("pdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x, probs/cdf[-1])
        plt.show()

    for idx in range(len(cdf)):
        cdf[idx] /= cdf[-1]

    if get_plots:
        fig2 = plt.figure()
        ax = fig2.add_subplot(111)
        plt.title("cdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x[:-1], cdf)
        plt.show() 
        
    if get_50_pt:
        # find 50% point
        cur_prob = 0
        idx = 0
        while(cur_prob < .5):
            cur_prob = cdf[idx]
            idx += 1
        fiftyfiftypt = x[idx]
        if do_print:
            print("50 50 change to be above or below")
            print(x[idx])


    #find expected value
    expected_value = 0
    for idx in range(len(cdf)):
        if idx == 0:
            expected_value += cdf[idx] * x[idx]
        else:
            cur_prob = cdf[idx - 1]
            next_prob = cdf[idx]
            actual_prob = next_prob - cur_prob
            expected_value += actual_prob * x[idx]
                                 
    if do_print:
        print("Expected Value")
        print(expected_value)
                                 
    if get_50_pt:
        return fiftyfiftypt, expected_value
    else:
        return expected_value

In [56]:
def test_preds(assets, baseline, pred_distance, df_test, df_train):
    rvs = get_rvs(baseline, df_train, assets, pred_distance)
    preds = {}
    actuals = {}
    for asset in assets:
        if asset != baseline:
            preds[asset] = []
            actuals[asset] = []
    
    pred_columns = []
    for col_idx, col in enumerate(df.columns):
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            pred_columns.append(col_idx)
            
            
    print('start')
    for idx, row in df_test.iterrows():
        for asset in assets:
            if asset != baseline:
#                 print(asset)
                col_name = asset + "_" + baseline  + "_" + str(pred_distance) + "dys_diff"
                actual = row[col_name]
                actuals[asset].append(actual)
#                 print(actual)
                columns = []
                pred_input = df_test.iloc[idx, pred_columns]
                if not pred_input.isnull().any():
                    prediction = predict(asset, baseline, rvs[asset], pred_input.tolist(), get_plots=False)
                    preds[asset].append(prediction)
                    # print(prediction)
                else:
                    preds[asset].append(None)
#                     print(None)
#                 print("---------")
        if idx % 10== 0:
            print(idx)

    return preds, actuals            

In [58]:
# inputs here

# path = 'C:\\Users\\plant\\
path = ''

baseline_asset = 'sp'

file_SP = 'SPY'  
div_SP = 'SPYDividend'
file_RE = 'IYR'
div_RE = 'IYRDividend'
file_BND = 'isharesBondIndexSince2003'
div_BND = 'USAggBondDividend'
file_EU = 'USD_EURHistoricalData'
div_EU = None
file_JPY = 'USD_JPYHistoricalData'
div_JP = None
file_GLD = 'GoldFuturesHistoricalData'
div_GLD = None
file_MID = 'IJH'
div_MID = 'IJHDividend'
file_SML = 'IJR'
div_SML = 'IJRDividend'
file_RUT = 'IWM'
div_RUT = 'IWMDividend'
file_EST = 'EZU'
div_EST = 'EZUDividend'
file_EMR = 'EEM'
div_EMR = 'EEMDividend'
file_JST = 'EWJ'
div_JST = 'EWJDividend'

# data_files = [file_name_SP, file_name_RE, file_name_BND, file_name_EU, file_name_JPY, file_name_GOLD, file_name_RUT]
# aliases = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']

# # input correlaries
# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut']
# cor_days_out = [20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 252, 252, 252, 252, 252, 252, 252, 60, 60, 60, 60, 60, 60, 60]
# pred_distance = 10
# assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']


data_files = [file_SP, file_RE, file_BND, file_EU, file_JP, file_GLD, file_MID, file_SML, file_RUT, file_EST, file_EMR, file_JST]
div_files = [div_SP, div_RE, div_BND, div_EU, div_JP, div_GLD, div_MID, div_SML, div_RUT, div_EST, div_EMR, div_JST]
aliases = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

# input correlaries
# cor_assets = ['sp', "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld",  "sp", "re", "bnd", "eu", "jp", "gld"]
# cor_days_out = [20,   20,    20,   20,   20,    20,   10,   10,   10, 10,   10,   10,   252,  252, 252, 252,  252,   252,    60,   60,    60,    60,    60,   60]
cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]
cor_days_out = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
pred_distance = 10
assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

In [59]:
df = load_data(path, data_files, aliases)


In [60]:
df.iloc[0:1]

,Date,sp,gld,jst,eu,jp,mid,sml,rut,re,est,emr,bnd
0,1990-01-02,NaN,404.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df = df.query(baseline_asset + ".notnull()")
df.reset_index(drop = True, inplace = True)

In [66]:
# add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets)
div_data = load_div_data(["sp", "re", "bnd", "eu", "jp", "gld"], [None, "IYRDividend.csv", None, None, None, None])
add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data)
add_pred_differences(pred_distance, baseline_asset, assets, df)

KeyError: 'mid_fut_10dys'

In [ ]:
df

In [ ]:
df.iloc[8060: 8061, [0, 1, 2, 3, 4, 5, 6, 7, 20]]

In [ ]:
df_test = df[(df['Date'] >= '2009-01-01') & (df['Date'] <= '2009-12-30')]
df_test = df_test.reset_index(drop=True)
df_train = df[(df['Date'] < '2009-01-01')]

In [ ]:
preds, actuals = test_preds(aliases, baseline_asset, pred_distance, df_test, df_train)

# import pickle 

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

with open('actuals.pkl', 'wb') as f:
    pickle.dump(actuals, f)



In [ ]:
import pickle
with open("preds.pkl",'rb') as f:
    preds = pickle.load(f)
    
with open("actuals.pkl",'rb') as f:
    actuals = pickle.load(f)

In [ ]:
sign_cor_neg = 0
sign_cor_pos = 0
incor_actual_neg = 0
incor_actual_pos = 0
total_diff = 0
total_count = 0
total_correct = 0

for pred, actual in zip(preds['gld'], actuals['gld']):
    if pred != None and not pd.isna(actual):
        if pred < 0 and actual < 0:
            sign_cor_neg += 1
            total_correct += 1
        elif pred > 0 and actual > 0:
            sign_cor_pos += 1
            total_correct += 1
        elif actual < 0:
            incor_actual_neg += 1
        else:
            incor_actual_pos +=1
        total_count += 1
        total_diff += abs(pred - actual)

In [ ]:
print("pred neg actual neg")
print(sign_cor_neg)
print("pred pos actual pos")
print(sign_cor_pos)
print("pred pos actual neg")
print(incor_actual_neg) 
print("pred neg actual pos")
print(incor_actual_pos )

print("-----")
print("total neg")
print(sign_cor_neg + incor_actual_neg)
print("total pos")
print(sign_cor_pos + incor_actual_pos)
print("----")
print("ave diff")
print(total_diff/total_count)
print("percent correct")
print(total_correct/total_count)


In [ ]:
total = 1
period_counts = 0

count = 0

test_asset = 're'
offset = 2

for date, pred, actual in zip(df_test["Date"][offset:], preds[test_asset][offset:], actuals[test_asset][offset:]):
    if pred != None and not pd.isna(actual):
        if count == 10:
            # if pred > .01:
            if pred > 0:
                total *= (1 + 10 * pred * actual)
                print("long: " + str(date) + ": " + str(total))
            # if pred < -.01:
            if pred < 0:
                total *= (1 + 10  * abs(pred) * -(actual))
                print("shorted: " + str(date) + ": " + str(total))
            period_counts += 1
    #         else:
    #             total *= (1 + actual)
            
            count = 0
        count += 1
        
        
print(test_asset)
print(total)
print(period_counts)